In [58]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams

import sys

import tensorflow as tf
from tensorflow import keras

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
from clean_helpers import *

take_full = False
test_locally = True
create_new_text_files = True

# Specify here what cleaning functions you want to use
cleaning_options = ['clean_new_line', 'remove_stopwords', 'clean_tags',
                    'clean_punctuation', 'remove_numbers', 'lemmatize', 'remove_saxon_genitive',
                    ]


clean = {
    "clean_new_line": clean_new_line,
    "lowercase": lowercase,
    "lemmatize": lemmatize,
    "remove_stopwords": remove_stopwords,
    "translate": perform_translation,
    "clean_punctuation": clean_punctuation,
    "clean_tags" : clean_tags,
    "remove_numbers": remove_numbers,
    "remove_saxon_genitive": remove_saxon_genitive,
    "gensim_simple": gensim_clean   # not a good idea to use it I think! It cleans everything which is not alphabetic (special char, numbers and so on)
}


# algorithm_used = ""
# algorithm = {
#     "naive_bayes": ,
#     "logistic_regression": ,
#     "svm": ,
#     "lstm":,
#     "fasttext":,
#     "cnn": ,
# }

# options = []
# additional_options = {
#     "count_frequency": ,
#     "count_ngrams": ,
    
# }

In [60]:
input_file_pos = 'Data/train_pos.txt'
if take_full:
    input_file_pos = 'Data/train_pos_full.txt'
  
input_file_neg = 'Data/train_neg.txt'
if take_full:
    input_file_neg = 'Data/train_neg_full.txt'
    
list_of_pos_sentences = []
with open(input_file_pos, 'r') as f:
    for line in f:
        list_of_pos_sentences.append(line)
 
list_of_neg_sentences = []
with open(input_file_neg, 'r') as f:
    for line in f:
        list_of_neg_sentences.append(line)

In [61]:
from data_handling import build_sentences

df = build_sentences(list_of_pos_sentences, list_of_neg_sentences)

print("unique words = {}".format(count_unique_words(df)))

unique words = 114427


In [62]:
# Perform all the cleaning options selected

for clean_option in cleaning_options:
    counter_of_occurrences = 0
    %time df = clean[clean_option](df)
    print(clean_option)
    print(df.head())
    print("unique words = {}".format(count_unique_words(df)))
    print("################################\n\n")
    

df.head()

CPU times: user 262 ms, sys: 6 µs, total: 262 ms
Wall time: 261 ms
clean_new_line
                                            sentence  label
0  <user> i dunno justin read my mention or not ....      1
1  because your logic is so dumb , i won't even c...      1
2  " <user> just put casper in a box ! " looved t...      1
3  <user> <user> thanks sir > > don't trip lil ma...      1
4  visiting my brother tmr is the bestest birthda...      1
unique words = 114427
################################


The number of scipy stopwords is 179
CPU times: user 597 ms, sys: 0 ns, total: 597 ms
Wall time: 597 ms
remove_stopwords
                                            sentence  label
0  <user> dunno justin read mention . justin god ...      1
1    logic dumb , even crop name photo . tsk . <url>      1
2  " <user> put casper box ! " looved battle ! #c...      1
3  <user> <user> thanks sir > > trip lil mama ......      1
4  visiting brother tmr bestest birthday gift eve...      1
unique words = 11425

,sentence,label
0,dunno justin read mention justin god know hope...,1
1,logic dumb even crop name photo tsk,1
2,put casper box ! looved battle ! #crakkbitch,1
3,thanks sir trip lil mama ... keep doin ya thang !,1
4,visiting brother tmr bestest birthday gift eve...,1


In [63]:
df_copy = df.copy()

df_copy['word'] = df_copy.sentence.apply(lambda x: x.split((" ")))

df_copy = df_copy.drop("sentence", axis=1)

df_exploded = df_copy.explode("word").reindex()

df_exploded = df_exploded.reset_index()

df_grouped = df_exploded.groupby("word").count().sort_values(by='index', ascending=False).reset_index()

df_non_alpha = df_grouped[df_grouped['word'].apply(lambda x: not x.isalpha())]

df_non_alpha.head(20)

,word,index,label
0,!,83074,83074
1,...,40967,40967
2,?,26418,26418
3,i'm,13656,13656
8,..,9456,9456
28,can't,4905,4905
32,*,4519,4519
36,<3,4361,4361
50,i'll,3205,3205
113,:d,1939,1939


In [64]:
count_unique_words(df)

100500

In [65]:
len(df['sentence'].apply(lambda x: x.split()).explode().unique())

100500

In [66]:
count_unique_ngrams(df, 3)

1010245

In [67]:
from helpers import create_labelled_file
k_folds = 5

In [70]:
from sklearn.feature_extraction.text import CountVectorizer

if test_locally:    
    # Create the bag of words
    # The token_pattern is used in order to avoid the preprocessor to remove special characters (like smiles)
    # or hashtags (Twitter!)
    vectorizer = CountVectorizer(vocabulary=df['sentence'].apply(lambda x: x.split()).explode().unique())  # The vectorizer is used to create the bag of words

    X = vectorizer.fit_transform(df['sentence'])
    Y = df['label']
    
    print(X.shape)

CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.28 s
(200000, 100500)


In [61]:
counter = 0
print(X.shape)
for x in X[0].toarray()[0]:
    if x > 0:
        counter += 1
print(counter)
print(df.iloc[0].sentence)
print(vectorizer.vocabulary_.get('#crakkbitch'))

(200000, 102408)
9
dunno justin read mention justin god know hope follow #believe
2723


In [10]:
# divide in train test split
# CAREFUL HERE: when working with neural nets, we need to convert -1, 1 labels into 0, 1
if test_locally:
    train_test_split = 0.7
    permut = np.random.permutation(X.shape[0])
    train_x = X[permut[: int(X.shape[0]*train_test_split)]]
    train_y = Y[permut[: int(X.shape[0]*train_test_split)]]
    
    test_x = X[permut[int(X.shape[0]*train_test_split):]]
    test_y = Y[permut[int(X.shape[0]*train_test_split):]]
    
    ## Convert all -1 into 0!
    train_y = train_y.where(train_y == 1, 0) 
    test_y = test_y.where(test_y == 1, 0)
    
    print(train_x.shape)
    print(test_x.shape)
    print(train_y)

(1750000, 403138)
(750000, 403138)
2253463    0
1388299    0
208807     1
1205350    1
1607799    0
          ..
1462319    0
1126134    1
331413     1
1876117    0
1804629    0
Name: label, Length: 1750000, dtype: int64


In [14]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
])

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
for iter in range(8):
    # train for 5 epochs the model 
    model.fit(train_x, train_y, epochs=1, batch_size=2048)
    # evaluate the test error
    model.evaluate(test_x[:10000],  test_y[:10000], verbose=2)

1750000/1750000 [==============================] - 3069s 2ms/sample - loss: 0.4354 - acc: 0.7983
 - 18s - loss: 0.4039 - acc: 0.8113
1750000/1750000 [==============================] - 3060s 2ms/sample - loss: 0.3783 - acc: 0.8249
 - 18s - loss: 0.3974 - acc: 0.8171
1750000/1750000 [==============================] - 3060s 2ms/sample - loss: 0.3455 - acc: 0.8431
 - 18s - loss: 0.3992 - acc: 0.8194
1750000/1750000 [==============================] - 3060s 2ms/sample - loss: 0.3156 - acc: 0.8584
 - 18s - loss: 0.4042 - acc: 0.8220
1750000/1750000 [==============================] - 3064s 2ms/sample - loss: 0.2885 - acc: 0.8722
 - 18s - loss: 0.4174 - acc: 0.8210
1750000/1750000 [==============================] - 3063s 2ms/sample - loss: 0.2651 - acc: 0.8838
 - 18s - loss: 0.4376 - acc: 0.8188
1750000/1750000 [==============================] - 3068s 2ms/sample - loss: 0.2455 - acc: 0.8935
 - 18s - loss: 0.4585 - acc: 0.8159
1750000/1750000 [==============================] - 3072s 2ms/sample -

In [ ]:
# done for 16 epochs
for iter in range(8):
    # train for 5 epochs the model 
    model.fit(train_x, train_y, epochs=1, batch_size=512)
    # evaluate the test error
    model.evaluate(test_x,  test_y, verbose=2)

1750000/1750000 [==============================] - 3194s 2ms/sample - loss: 0.4105 - acc: 0.8075
 - 1420s - loss: 0.4153 - acc: 0.8047
1750000/1750000 [==============================] - 3295s 2ms/sample - loss: 0.4092 - acc: 0.8082
 - 1385s - loss: 0.4143 - acc: 0.8052
1750000/1750000 [==============================] - 3415s 2ms/sample - loss: 0.4080 - acc: 0.8089
 - 1371s - loss: 0.4133 - acc: 0.8056
1750000/1750000 [==============================] - 3132s 2ms/sample - loss: 0.4068 - acc: 0.8095
 - 1388s - loss: 0.4125 - acc: 0.8059
1750000/1750000 [==============================] - 3118s 2ms/sample - loss: 0.4057 - acc: 0.8101
 - 1331s - loss: 0.4117 - acc: 0.8064
1750000/1750000 [==============================] - 3092s 2ms/sample - loss: 0.4046 - acc: 0.8107
 - 1337s - loss: 0.4110 - acc: 0.8071
 881664/1750000 [==============>...............] - ETA: 27:08 - loss: 0.4031 - acc: 0.8116

In [18]:
model.save("model_big_16_epochs.model")

In [96]:
test_loss, test_acc = model.evaluate(test_x,  test_y, verbose=2)

 - 13s - loss: 0.4323 - acc: 0.7965


In [ ]:
if test_locally:
    # Do cross validation on the bag of words, using the neural network
    df_precisions = {}
    for epochs in tqdm(range(10, 20, 2)):
        precisions = []
        for k in range(k_folds):
            
        df_precisions[epochs] = precisions
        print(np.array(precisions).mean())